# ___Les Options___

---

**Attention, bien que ce chapitre traite essentiellement des Option et des Result, il introduit aussi la notion de type generique. Ce concept sera détaillé plus tard dans le cours.** 

> Dans nombre de langage informatique, il arrive que nous ayons à donner certaines valeurs à des variables pour exprimer des erreurs, qu'elles ne contiennent rien ou bien tout simplement qu'elles ne sont pas utilisables pour le moment. Généralement, on leur donnera comme valeur -1 ou 0 ou bien un pointeur nul en C. Or cette bidouille est source de nombreuses erreurs puisqu'il suffit d'oublier un test quelque part dans le code ou bien de se rendre compte que la valeur que l'on a donnée comme erreur est en fait valide dans certains cas pour que le programme plante ou ne semble pas logique.

> En s'inspirant de ce qui se fait dans les langages fonctionnels, les développeurs de Rust ont décide de créer deux énumérations intégrées a la std afin de pouvoir gérer ce problème. La première énumération est l'`Option` et la seconde est le `Result`.

## Le prototype de Option

- Un Option est une énumération de la std qui possède deux variant :
> - Un variant qui exprime qu'il y a quelque chose, le variant `Some`
> - Un variant qui exprime qu'il n'y a rien du tout, le variant `None`

Ce "quelque chose" peut être n'importe quel type, on pourrait ainsi tout autant vouloir une Option d'un usize ou bien une option d'une structure. Comme ll serait illogique et fastidieux d'avoir a créer une Option manuellement pour chacun des types de langages sans compter nos propres types, les Option utiliseront un type générique noté `T` . Ce type générique T sera défini par le programmeur lorsqu'il créera sa variable de type Option.

![PROTO](pictures/proto_options.png)

> Par exemple si l'on veut une Option de usize, on remplacera T par usize soit `Option<usize>` et Some(T) sera remplace par le compilateur avec `Some(usize)`.

## Création et utilisation d'une Option

- Commençons par un exemple très simple :

In [3]:
let opt: Option<usize> = Some(42);
dbg!(opt);

[src/lib.rs:105] opt = Some(
    42,
)


> Il n'a rien d'étonnant au fait que nous avons fait la même chose que pour une énumération puisque Option est une énumération. L'unique différence avec ce qui a été vu au chapitre précédent, c'est le type générique que nous avons spécifie ici entre `<` et `>`.

- Il est donc aussi tout à fait possible de pouvoir utiliser un match sur l'Option.

In [7]:
fn check_option(opt: Option<usize>) {
    match opt {
        Some(value) => println!("1: The Option contains an usize. The value is {}.", value),
        None => println!("2: The Option contains nothing !"),
    }
}
check_option(Some(42));
check_option(None);

let opt: Option<f64> = Some(3.15);
if let Some(f) = opt {
    println!("The contained value is {}", f);
}

1: The Option contains an usize. The value is 42.
2: The Option contains nothing !
The contained value is 3.15


()

> On remarquera aussi que contrairement à nos énumérations du chapitre précédent, nous n'avons pas à expliciter le type Option pour les variants. Car, afin d'éviter d'avoir a imposer au codeur d'écrire Option::Some(value), Option::None, ou encore use Option::\*\*, les développeurs de Rust ont décide d'intégrer les variants de Option au prélude, **le prélude désigne tout ce qui est importes par défaut.**

## Les méthodes de Option

### Unwrap

- Il existe une pléthore de méthodes pour Option, la plus connue est certainement unwrap() qui fera paniquer le programme si l'Option est égal a None ou bien qui retournera le contenu de Some s'il existe.

![UNWRAP_OPT](pictures/unwrap_options.png)

- Bien que nous n'avons pas encore vu l'allocation dynamique, on peut imaginer cet exemple. Je possède une collection de 3 éléments et j'utilise une fonction qui me permet de récupérer un de ses éléments en le retirant de ma collection. Lors de mon 4e appel, ma collection étant vide, je recevrai None ! Voyons ce code :

In [21]:
let mut v = vec![11, 22, 44]; // Creation d'une collection de 3 elements
let elem1 = v.pop(); // Recuperation du troisieme element
dbg!(elem1); // elem1 vaut Some(44)
dbg!(v.pop()); // Recuperation du second...
dbg!(v.pop()); // Recuperation du premier...
// A ce point, ma collection est vide !
let elemX = v.pop(); // On tente de recuperer un autre element !
dbg!(elemX); // elemX vaut None

[src/lib.rs:118] elem1 = Some(
    44,
)
[src/lib.rs:119] v.pop() = Some(
    22,
)
[src/lib.rs:120] v.pop() = Some(
    11,
)
[src/lib.rs:123] elemX = None


- Il peut arriver que la présence d'aucun élément soit une chose qui n'est jamais censée arriver dans un programme et qui signifie que quelque chose en amont s'est mal passée... Dans ce cas de figure, unwrap() qui peut faire paniquer le programme est une méthode de choix :

In [14]:
let mut v = vec![12]; // Creation d'une collection de 1 element
let elem = v.pop(); // elem vaut Some(12)
let unwrap_result = elem.unwrap(); // Unwrap retourne T si l'option vaut Some(T)
dbg!(unwrap_result);

let elem = v.pop(); // Recuperation d'un nouvel element. Ici ce sera donc None
let unwrap_result = elem.unwrap(); // Comme elem vaut None, mon code paniquera !

[src/lib.rs:117] unwrap_result = 12
thread '<unnamed>' panicked at 'called `Option::unwrap()` on a `None` value', src/lib.rs:120:26
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:114:5
   2: core::panicking::panic
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


### Map

- Bien que cette méthode soit assez complexe pour l'instant puisqu'elle utilise deux types génériques et une closure (cf: prog fonctionnelle), il est nécessaire de la présenter puisqu'on la croise tout le temps en Rust. Voici son prototype pour Option :
    
![MAP_OPT](pictures/map_options.png)
    
> Pour faire simple, map transforme une Option d'un type en une Option d'un autre type grâce a une fonction de conversion. Par contre, si l'Option était a None, la sortie de map sera toujours None.

- Voici un exemple plus simple avec des types explicites et connus et sans closure :

In [29]:
fn usize_to_u64(v: usize) -> u64 {
    v as u64
}
let opt_usize: Option<usize> = Some(42);
let opt_u64: Option<u64> = opt_usize.map(usize_to_u64);
dbg!(opt_u64);

[src/lib.rs:128] opt_u64 = Some(
    42,
)


> Map a permis de transformer l'Option de usize en une Option de u64

**Il existe encore beaucoup de méthodes pour Option\<T\>. Voir la documentation de la std :**

**Enum std::option::Option**

## Aller plus loin avec les Options

* Les Options sont aussi utilises par les Iterateurs que nous avons déjà brièvement croise dans le **chapitre 004 Control Flow de 002 Basics**. Parcourir un Iterateur retourne Some(quelque chose) tant qu'il y a des éléments à parcourir puis None quand il n'y a plus d'élément.

In [36]:
let fizzbuzz = ["Fizz", "Buzz", "FizzBuzz"];
for word in fizzbuzz {
    println!("{}", word);
}

Fizz
Buzz
FizzBuzz


()

> Derrière la boucle **for** se cache des iterateurs et des Options. Le code ci-dessous se comporte de la même façon
 :
> ```
> let fizzbuzz = &["Fizz", "Buzz", "FizzBuzz"];
> let mut iter = fizzbuzz.iter(); // Creation d'un iterateur sur fizzbuzz
> while let Some(word) = iter.next() {
>     println!("{}", word);
> }
> ```

*Les Options constituent un des piliers centraux du langage Rust, on les retrouve partout dans la std et donc dans tous les programmes en Rust.*

## Exercices

### Pourquoi le code ci-dessous ne peut pas compiler ? Quelle correction appliquer ?

In [37]:
let v: Option<usize> = 42;
if let 42 = v {
    println!("OK");
}

Error: mismatched types

Error: mismatched types

Le contenu de l'Option n'est pas 42 mais Some(42), le compilateur ne comprend pas notre comparaison de motif entre un usize et une Option de usize.

```
let v: Option<usize> = 42;
if let Some(42) = v {
    println!("OK");
}
```

### En vous aidant de la documentation de take et de unwrap pour Option, l'exécution de ce code se passe-t-elle correctement ? Si oui, pourquoi ?

```
let mut v: Option<usize> = Some(42);
let value = v.take().unwrap();
dbg!(value);
dbg!(v.unwrap());
```

Le déroulement du code était correct durant les 3 premières lignes. Seulement, en utilisant take(), j'ai récupéré 42 de mon Option qui est devenu None, ainsi lors de la 4e ligne, unwrap() fera paniquer le code.